In [1]:
encoder_model = 'GLSTM' # BLSTM, TLSTM, TCNN, GLSTM
includeLabel = True # delexicalization
triple_mode = 'graph' # single, graph
auto_adaptive = True # attention model

if triple_mode == 'single':
    triple_max_len = 20
    input_max_len = 140
    output_max_len = 70
elif triple_mode == 'graph':
    triple_max_len = 5
    input_max_len = 140
    output_max_len = 70
    
dataset_test = 'test_unseen' # test, test_unseen, test_gkb
model_num = 20 # num epoch saved

from config import load_parameters
from model_zoo import TranslationModel
import utils
from keras_wrapper.cnn_model import loadModel
from keras_wrapper.dataset import loadDataset
from keras_wrapper.extra.callbacks import PrintPerformanceMetricOnEpochEndOrEachNUpdates
params = load_parameters()
dataset = loadDataset('datasets/Dataset_GKB_Data.pkl')

dataset.setInput('data/'+dataset_test+'.src',
            'test',
            type='text',
            id='source_text',
            pad_on_batch=False,
            tokenization='tokenize_none',
            fill='end',
            max_text_len=input_max_len,
            min_occ=0)

dataset.setInput(None,
            'test',
            type='ghost',
            id='state_below',
            required=False)

Using Theano backend.
Using cuDNN version 6021 on context None
Mapped name None to device cuda0: Tesla K40c (0000:03:00.0)
[02/10/2019 21:24:10] <<< Loading Dataset instance from datasets/Dataset_GKB_Data.pkl ... >>>
[02/10/2019 21:24:11] <<< Dataset instance loaded >>>
[02/10/2019 21:24:11] 	Applying tokenization function: "tokenize_none".
[02/10/2019 21:24:11] Loaded "test" set inputs of type "text" with id "source_text" and length 891.
[02/10/2019 21:24:11] Loaded "test" set inputs of type "ghost" with id "state_below" and length 891.


In [2]:
params['INPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len['all_vocab']
params['OUTPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len['all_vocab']
params['INPUT_MAX_LEN'] = input_max_len
params['OUTPUT_MAX_LEN'] = output_max_len
params['TRIPLE_MAX_LEN'] = triple_max_len
params['ENCODER_MODEL'] = encoder_model
params['AUTO_ADAPTIVE'] = auto_adaptive

nmt_model = TranslationModel(params,
                             model_type='GroundHogModel', 
                             model_name='tutorial_model',
                             vocabularies=dataset.vocabulary,
                             store_path='trained_models/gkb_model_/',
                             verbose=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
source_text (InputLayer)        (None, 140)          0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 5)            0           source_text[0][0]                
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 5)            0           source_text[0][0]                
__________________________________________________________________________________________________
source_word_embedding (Embeddin (None, 5, 300)       2855700     lambda_2[0][0]                   
                                                                 lambda_4[0][0]                   
          

__________________________________________________________________________________________________
target_word_embedding (Embeddin (None, None, 300)    2855700     state_below[0][0]                
__________________________________________________________________________________________________
initial_state (Dense)           (None, 512)          524800      masked_mean_1[0][0]              
__________________________________________________________________________________________________
initial_memory (Dense)          (None, 512)          524800      masked_mean_1[0][0]              
__________________________________________________________________________________________________
state_below_gaussian_noise (Gau (None, None, 300)    0           target_word_embedding[0][0]      
__________________________________________________________________________________________________
initial_state_gaussian_noise (G (None, 512)          0           initial_state[0][0]              
__________

target_text (TimeDistributed)   (None, None, 9519)   2865219     out_layer_linear_0_dropout[0][0] 
Total params: 28,324,281
Trainable params: 28,317,185
Non-trainable params: 7,096
__________________________________________________________________________________________________


In [3]:
model_name = nmt_model.model_path[:-2] + "/epoch_" + str(model_num)
nmt_model.model.load_weights(model_name + '_weights.h5')
nmt_model.model_init.load_weights(model_name + '_weights_init.h5')
nmt_model.model_next.load_weights(model_name + '_weights_next.h5')

In [4]:
inputMapping = dict()
for i, id_in in enumerate(params['INPUTS_IDS_DATASET']):
    pos_source = dataset.ids_inputs.index(id_in)
    id_dest = nmt_model.ids_inputs[i]
    inputMapping[id_dest] = pos_source
nmt_model.setInputsMapping(inputMapping)

outputMapping = dict()
for i, id_out in enumerate(params['OUTPUTS_IDS_DATASET']):
    pos_target = dataset.ids_outputs.index(id_out)
    id_dest = nmt_model.ids_outputs[i]
    outputMapping[id_dest] = pos_target
nmt_model.setOutputsMapping(outputMapping)

In [5]:
params_prediction = {'max_batch_size': 64,
                     'n_parallel_loaders': 8,
                     'predict_on_sets': ['test'],
                     'beam_size': 12,
                     'maxlen': output_max_len,
                     'model_inputs': ['source_text', 'state_below'],
                     'dataset_inputs': ['source_text', 'state_below'],
                     'normalize': True, 
                     'alpha_factor': 0.6                   
                     }
predictions = nmt_model.predictBeamSearchNet(dataset, params_prediction)['test']

[02/10/2019 21:24:27] WARNING: parallel loaders are not implemented
[02/10/2019 21:24:27] <<< Predicting outputs of test set >>>



Sampling 891/891  -  ETA: 2s    
 Total cost of the translations: 5027.274233 	 Average cost of the translations: 5.642283
The sampling took: 2583.798322 secs (Speed: 2.899886 sec/sample)


In [6]:
from keras_wrapper.utils import decode_predictions_beam_search
vocab = dataset.vocabulary['all_vocab']['idx2words']
predictions = decode_predictions_beam_search(predictions,
                                             vocab,
                                             verbose=params['VERBOSE'])

[02/10/2019 22:07:31] Decoding beam search prediction ...


In [7]:
filepath = nmt_model.model_path+'' + dataset_test + '_sampling.pred'  # results file
from keras_wrapper.extra.read_write import list2file
list2file(filepath, predictions)

In [8]:
import os
import shutil
fw = open('evaluation.sh', 'w')

fw.write('python relex.py '+filepath+' \n')

fw.write('perl multi-bleu.perl \
    data/eval/' + dataset_test+'/* \
    < '+filepath+'.final \
    > '+filepath+'.BLEU \n')
    
fw.close()

In [9]:
! bash evaluation.sh